# **ch.05 트리 알고리즘**  
## 05-2 교차 검증과 그리드 서치  
### 풀어야 할 문제:  
> ###  model의 generalization performance를 확인하기 위해 test set을 사용하지 않고 model의 hyperparameter를 tuning하라
### *검증 세트*  
지금까지 model의 general performance(일반화 성능)를 확인하기 위해 test set을 사용했었음. 그러나 test set을 반복해서 사용하면 점점 test set에 맞추는 셈이 됨  
$\therefore$ test set으로 model의 generl performance를 확인하기 위해서는 가능한 한 test set을 사용하지 말아야 함


그렇다면 어떻게 hyperparameter를 tuning할까?  
가장 간단한 방법은 train set에서 test set이외에 또 data를 나누는 것  
이를 validation data라고 함


train set으로 model을 훈련하고 validation set으로 모델을 평가하여 hyperparameter를 tuning함  
그 후에 tuned hyperparameter를 사용하여 train set과 validation set을 합쳐 train하고 test set으로 final score를 평가


wine data를 준비하여 validation set 만들기  
+ data 준비하기

In [1]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

+ input data(feature data), target data 만들기

In [2]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

+ train set, test set 나누기  (test size=0.2)

In [3]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

+ train set에서 validation set 나누기 (validation size=0.2)

In [4]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

train set, validation set의 크기 확인

In [5]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


train set과 validation set으로 train model, score model

In [6]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)

dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


### *교차 검증*  
validation set을 만드느라 train set이 줄었음  
보통 많은 data를 train에 사용할수록 좋은 model을 만들 수 있음  
그렇다고 validation set을 너무 조금 만들면 validation score의 변동폭이 커지고 불안정해짐  
이런 경우에 cross validation을 이용하면 안정적인 score를 얻을 수 있고, train에 더 많은 data를 사용할 수 있음  


cross validation은 validation set을 떼어 내어 평가하는 과정을 여러 번 반복한 후, score를 평균하여 fianl validation score를 얻음  
cross validation을 활용하면, 
+ data의 80~90%까지 train에 사용할 수 있음
+ validation set이 줄어 각 fold(iteration 1회)의 score는 불안정해 질 수 있지만, 각 fold의 score를 mean하기에 안정적인 score로 판단할 수 있음


scikit-learn은 cross_validate() cross validation function을 제공  
사용법: 
+ 평가할 model object를 factor로 전달,  
+ validation set을 떼어내지 않고 entire train set을 전달
+ cv parameter에 전달하는 splitte의 n_splits parameter에 진행할 fold 수를 지정 (base: 5)

cross_validate() fuction은 'fit_time, score_time, test_score'의 key를 가진 dictionary를 return  
+ fit_time: model을 train하는 시간
+ score_time: validation을 진행하는 시간
+ test_score: 각 fold의 score (final mean score가 아님)


cross_validate() function을 활용하여 score model

In [7]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)

print(scores)

{'fit_time': array([0.00496626, 0.00441599, 0.00434685, 0.00422406, 0.00409412]), 'score_time': array([0.00043893, 0.0003221 , 0.00027514, 0.00027108, 0.00029373]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


'test_score' key의 value를 mean하여 final score 확인

In [8]:
import numpy as np

np.mean(scores['test_score'])

0.855300214703487

cross_validate()는 train set을 섞어 fold를 나누지 않음  
$\therefore$ data를 섞기 위해서는 splitter(분할기)를 지정해야 함 (cv parameter에 splitter 전달)


splitter는 cross validation에서 fold를 어떻게 나눌지 결정  
cross_validate()는 기본적으로 regression model의 경우 KFod splitter를 사용하고, classification model의 경우 StratifiedKFold를 사용(target class를 골고루 나누기 위해)


$\therefore$ 다음의 splitter를 지정한 code는 앞선 splitter를 지정하지 않은 code와 동일한 작업을 수행

In [9]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())

print(scores)

{'fit_time': array([0.00525308, 0.00453401, 0.00442982, 0.00431323, 0.00418019]), 'score_time': array([0.00055099, 0.0003531 , 0.0002811 , 0.00032902, 0.0002799 ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


train set을 섞고, 10-fold cross validation

In [10]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

scores = cross_validate(dt, train_input, train_target, cv=splitter)

print(scores, np.mean(scores['test_score']))

{'fit_time': array([0.00611925, 0.00520182, 0.00488591, 0.00484276, 0.00461006,
       0.00469494, 0.00485706, 0.00464797, 0.00466084, 0.004601  ]), 'score_time': array([0.00043583, 0.00026011, 0.00019789, 0.00022602, 0.00018311,
       0.00018311, 0.0002079 , 0.00017977, 0.00017309, 0.00017285]), 'test_score': array([0.83461538, 0.87884615, 0.85384615, 0.85384615, 0.84615385,
       0.87307692, 0.85961538, 0.85549133, 0.85163776, 0.86705202])} 0.8574181117533719


### *하이퍼파라미터 튜닝*  
model parameter: model이 learnin하는 parameter  
hyperparameter: model이 학습할 수 없어 사용자가 지정해야 하는 parameter


hyperparameter tuning  
base value로 model을 train하고, validation set의 score나 cross validation을 통해 parameter를 조금씩 바꿈

hyperparameter tuning은 각각의 parameter에 대해 순차적으로 진행하는 것이 아니라, 모든 parameter에 대해 동시적으로 진행해야 함  
$\because$ parameter끼리 영향을 주고받음  
이러한 과정은 매우 복잡하기 때문에 scikit-learn에서 제공하는 grid search를 사용


scikit-learn의 GridSearchCV class는 hyperparameter searching과 cross validation을 한 번에 수행  
&rarr; 따로 cross_validate() function을 사용할 필요가 없음  


예제를 통해 practice grid search  
기본 parameter를 사용한 decision tree에서 min_impurity_decrease parameter의 최적값 찾기
+ import GridSearchCV
+ search할 parameter와 serch할 value의 목록을 dictionary로 만들기

In [11]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

+ generate GridSearchCV object


GridSearchCV의 cv parameter의 basic value는 5  
$\therefore$ min_impurity_decrease value마다 5-fold cross validation 수행  
많은 model(5\*5=25)을 train하기 때문에 parallel(병렬) 실행할 cpu core 수를 지정하는 n_jobs parameter를 -1로 지정하여 모든 core 사용

In [12]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

+ fit() method로 grid search 진행

In [13]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

scikit-learn의 grid search는 train이 끝나면 25개의 model 중에서 validation score가 가장 높은 model의 parameter 조합으로 train set에서 다시 model을 train함  
이 model은 class object의 best_estimator_ attribute에 저장되어 있음


best_estimator_의 model 평가

In [14]:
dt = gs.best_estimator_

print(dt.score(train_input, train_target))

0.9615162593804117


grid search로 찾은 최적의 parameters는 best_params_ attribute에 저장되어 있음


best_params_ 확인

In [15]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


각 parameter에서 수행한 cross validation의 평균 score는 cv_results_ attribute의 'mean_test_score'key의 value로 저장되어 있음


score 확인

In [16]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


min_impurity_decrease에 max_depth, min_samples_split parameter도 tuning하기


serch할 parameter와 value를 dictionary로 만들기  
+ np.arange() function은 첫 번째 parameter value에서 두 번째 parameter에 도달할 때까지 세 번째 parameter를 계속 더한 array를 만듦
+ range() function은 integer scale에서 np.arange() function과 비슷한 작업 수행

In [17]:
params = {'min_impurity_decrease':np.arange(0.0001, 0.001, 0.0001), 'max_depth':range(5, 20, 1),'min_samples_split':range(2, 100, 10)}

이 parameter로 수행할 cross validation은 9*15*10=1350 회이고, 각 5-fold cross validation을 수행하므로 만들어지는 model은 1350*5=6750개


n_jobs parameter를 -1로 지정하여 grid search 실행

In [18]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

최적의 pararmeter 조합 확인

In [19]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


최상의 cross validation score 확인

In [20]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


grid search로 최적의 parameter 조합을 찾아내기는 했지만, parameter value의 간격은 임의로 정한 것, 이보다 조금 더 좁거나 넓은 간격으로 시도해 볼 수는 없을까?


### *랜덤 서치*  
parameter value가 수치일 때, value의 범위나 간격을 미리 정하기 어려움  
또한 너무 많은 parameter가 있어 grid search 시간이 오래 걸릴 수 있음  
이 때 유용한 것이 random search


random search에는 parameter value의 목록을 전달하는 것이 아니라 parameter를 sampling할 수 있는 probability distribution을 전달


probability distribution 알아보기  
+ scipy의 probability distribution class를 import

In [21]:
from scipy.stats import uniform, randint

uniform: 주어진 실수 범위에서 고르게 value를 뽑는 기능 수행 (균등 분포(uniform distribution)에서 sampling)  
randint: 주어진 정수 범위 uniform distribution에서 sampling


+ 0~10 사이의 범위를 갖는 randint object를 만들고 10개의 숫자를 sampling

In [22]:
rgen = randint(0, 10)
rgen.rvs(10)

array([2, 4, 7, 8, 8, 4, 7, 0, 8, 2])

10개 밖에 sampling하지 않았기에 고르지는 않아 보이지만, 수를 늘리면 고르다는 것을 확인할 수 있음


+ 1000개를 sampling해서 각 숫자의 개수 세기

In [23]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 96,  88,  96, 101, 104, 127, 105, 115,  84,  84]))

+ 0~1 사이에서 10개 실수 sampling

In [24]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.5694066 , 0.84175432, 0.07334713, 0.0394574 , 0.10817733,
       0.72033466, 0.11231128, 0.21322461, 0.55766991, 0.12842676])

두 class는 난수 발생기와 유사  
random search에 randint와 uniform object를 넘겨주고 총 몇 번을 sampling할 지 지정할 수 있음  
sampling 횟수는 system resource가 허용하는 범위 내에서 최대한 크게 하는 것이 좋음


search할 parameter dictionary 만들기  
leaf node가 되기 위한 minimum sample 개수를 지정하는 min_samples_leaf parameter도 search 대상에 추가

In [25]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
         'max_depth': randint(20, 50),
         'min_samples_split': randint(2, 25), 
         'min_samples_leaf': randint(1, 25)
         }

random search class인 RandomizedSearchCV를 import,  
n_iter parameter에 sampling할 횟수(100) 전달,  
random search 수행

In [26]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)

gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x13cf8d7d0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x13cf9bad0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x13cf86250>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x13cf852d0>},
                   random_state=42)

결과 확인  
+ 최적의 parameter 조합 확인

In [27]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


+ 최고의 cross validation score 확인

In [28]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


+ 최적의 model을 활용하여 test set에 대한 score 확인

In [29]:
dt = gs.best_estimator_

dt.score(test_input, test_target)

0.86

test set에 대한 score가 validation set보다 조금 낮은 것이 일반적  
score가 아주 만족스럽지는 않지만, test set을 활용하지 않고 최적의 parameter를 찾아냄